In [10]:
using Test

In [11]:
f = readlines("day16.input");

In [12]:
function parse_input(input)
    flows = Dict{String, Int64}()
    edges = Dict{String, Vector{String}}()
    for line in input
        name_re = r"Valve ([A-Z][A-Z]) has flow rate=\d+; tunnels? leads? to valves? .*"
        flow_re = r"Valve [A-Z][A-Z] has flow rate=(\d+); tunnels? leads? to valves? .*"
        edge_re = r"Valve [A-Z][A-Z] has flow rate=\d+; tunnels? leads? to valves? (.*)"
        name = first(match(name_re, line).captures)
        flows[name] = parse(Int64, first(match(flow_re, line).captures))
        edges[name] = [strip(x) for x in (split(first(match(edge_re, line).captures), ","))]
    end
    flows, edges
end

parse_input (generic function with 1 method)

In [13]:
subproblems = Dict()

Dict{Any, Any}()

In [14]:
function step(pos, time, open, adjacent, flows)
    global subproblems

    next_gen_problems = []
    score = subproblems[(pos, time, open)]

    for adj in adjacent
        push!(next_gen_problems, (adj, time+1, open))
        if (adj, time+1, open) in keys(subproblems)
            subproblems[(adj, time+1, open)] = max(score, subproblems[(adj, time+1, open)])
        else
            subproblems[(adj, time+1, open)] = score
        end
    end    
    if pos ∉ open && flows[pos] > 0
        open = copy(open)
        push!(open, pos)
        
        push!(next_gen_problems, (pos, time+1, open))
        if (pos, time+1, open) in keys(subproblems)
            subproblems[(pos, time+1, open)] = max(score + flows[pos]*(30-time), subproblems[(pos, time+1, open)])
        else
            subproblems[(pos, time+1, open)] = score + flows[pos]*(30-time)
        end
    end

    next_gen_problems
end

step (generic function with 1 method)

In [15]:
function solve_part_1(input)
    flows, edges = parse_input(input)

    position = "AA"
    time = 30
    
    global subproblems
    subproblems = Dict()
    subproblems[("AA", 1, Set{String}())] = 0

    problems = Set{Tuple{String, Int64, Set{String}}}()
    problems2 = Set{Tuple{String, Int64, Set{String}}}()
    push!(problems, first(keys(subproblems)))

    for i=1:30
        for problem in problems
            p = step(problem[1], i, problem[3], edges[problem[1]], flows)
            push!(problems2, p...)
        end

        problems = problems2
        problems2 = Set{Tuple{String, Int64, Set{String}}}()
    end

    # for problem in keys(subproblems)
    #     if problem[2] == 6 && subproblems[problem] > 500
    #         println("$problem : $(subproblems[problem])")
    #     end
    # end

    maximum(values(subproblems))
end

solve_part_1 (generic function with 1 method)

In [16]:
@test solve_part_1(String.(split("Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II", "\n"))) == 1651


Test Passed

In [17]:
solve_part_1(f)

2056